# Ames Housing — Notebook A2  
## Missing Value Strategy (Feature-Aware, Regression Setup)

This notebook handles missing values **correctly**:
- Different features → different strategies
- No blanket mean/median filling
- Decisions driven by feature meaning

No encoding, no scaling, no modeling yet.


# 1️⃣ Load Dataset (Clean State)

In [1]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

DATA_PATH = "AmesHousing.csv"  # update if needed
df = pd.read_csv(DATA_PATH)

df.shape


(2930, 82)

# 2️⃣ Snapshot Missing Values (Before Any Fix)

In [2]:
missing_before = (
    df.isna()
      .sum()
      .to_frame("missing_count")
      .assign(missing_percent=lambda x: (x["missing_count"] / len(df) * 100).round(2))
      .sort_values(by="missing_percent", ascending=False)
)

missing_before[missing_before["missing_count"] > 0].head(20)


,missing_count,missing_percent
Pool QC,2917,99.56
Misc Feature,2824,96.38
Alley,2732,93.24
Fence,2358,80.48
Mas Vnr Type,1775,60.58
Fireplace Qu,1422,48.53
Lot Frontage,490,16.72
Garage Cond,159,5.43
Garage Finish,159,5.43
Garage Yr Blt,159,5.43


## 3️⃣ Structural Missingness — Categorical Features

(Missing = Feature Not Present)

In [3]:
structural_cat_features = [
    "Pool QC",
    "Misc Feature",
    "Alley",
    "Fence",
    "Fireplace Qu",
    "Garage Type",
    "Garage Finish",
    "Garage Qual",
    "Garage Cond",
    "Bsmt Qual",
    "Bsmt Cond",
    "Bsmt Exposure",
    "BsmtFin Type 1",
    "BsmtFin Type 2"
]

df[structural_cat_features] = df[structural_cat_features].fillna("None")


## 4️⃣ Structural Missingness — Numeric (Presence-Based)

In [4]:
structural_num_features = [
    "Garage Yr Blt",
    "Garage Area",
    "Garage Cars",
    "BsmtFin SF 1",
    "BsmtFin SF 2",
    "Bsmt Unf SF",
    "Total Bsmt SF"
]

df[structural_num_features] = df[structural_num_features].fillna(0)


## 5️⃣ Lot Frontage — Neighborhood-wise Median Imputation

In [4]:
# Compute median frontage per neighborhood
lot_frontage_median = (
    df.groupby("Neighborhood")["Lot Frontage"]
      .median()
)

# Apply neighborhood-wise median
df["Lot Frontage"] = df.apply(
    lambda row: lot_frontage_median[row["Neighborhood"]]
    if pd.isna(row["Lot Frontage"]) else row["Lot Frontage"],
    axis=1
)


## 6️⃣ Masonry Veneer — Ambiguous Missingness (Handled Carefully)

In [6]:
# Mas Vnr Type: missing likely means "None"
df["Mas Vnr Type"] = df["Mas Vnr Type"].fillna("None")

# Mas Vnr Area: if type is None, area should be 0
df.loc[df["Mas Vnr Type"] == "None", "Mas Vnr Area"] = 0


## 7️⃣ Verify Missing Values (After Fix)

In [7]:
missing_after = (
    df.isna()
      .sum()
      .to_frame("missing_count")
      .assign(missing_percent=lambda x: (x["missing_count"] / len(df) * 100).round(2))
      .sort_values(by="missing_percent", ascending=False)
)

missing_after[missing_after["missing_count"] > 0].head(10)


,missing_count,missing_percent
Lot Frontage,3,0.10
Bsmt Full Bath,2,0.07
Bsmt Half Bath,2,0.07
Electrical,1,0.03


## 8️⃣ Sanity Check — No Target Leakage

In [8]:
df["SalePrice"].isna().sum()

np.int64(0)

## 9️⃣ Final Dataset Shape After Missing Value Handling

In [9]:
df.shape


(2930, 82)

In [10]:
df[["Neighborhood", "Lot Frontage"]].sample(5)

,Neighborhood,Lot Frontage
1521,SWISU,51.0
869,CollgCr,90.0
139,NAmes,70.0
1818,Sawyer,64.0
1047,NPkVill,24.0


## Final Missing-Value Fix as the data is very less, so no spatial issue and we can go with median here.

In [11]:
# Final numeric median imputation (very small leftovers)
df["Lot Frontage"] = df["Lot Frontage"].fillna(df["Lot Frontage"].median())
df["Bsmt Full Bath"] = df["Bsmt Full Bath"].fillna(df["Bsmt Full Bath"].median())
df["Bsmt Half Bath"] = df["Bsmt Half Bath"].fillna(df["Bsmt Half Bath"].median())

# Final categorical mode imputation
df["Electrical"] = df["Electrical"].fillna(df["Electrical"].mode()[0])

In [12]:
df.isna().sum().sum()

np.int64(0)

In [13]:
# =========================================================
# FINAL STEP: Persist Cleaned Dataset for Downstream Use
# =========================================================

# 1. Sanity check: no missing values should remain
total_missing = df.isna().sum().sum()
print("Total missing values remaining:", total_missing)

if total_missing != 0:
    raise ValueError(
        "Missing values still present! Do NOT proceed to next notebook."
    )

# 2. Optional: quick dataset snapshot
print("Final dataset shape after cleaning:", df.shape)

# 3. Save cleaned dataset
CLEAN_DATA_PATH = "ames_housing_cleaned_v1.csv"
df.to_csv(CLEAN_DATA_PATH, index=False)

print(f"Cleaned dataset saved to: {CLEAN_DATA_PATH}")


Total missing values remaining: 0
Final dataset shape after cleaning: (2930, 82)
Cleaned dataset saved to: ames_housing_cleaned_v1.csv
